<a href="https://colab.research.google.com/github/haleelsada/Cryptweets/blob/main/Cryptweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cryptweet**

**Integrating Twitter API with transformers to predict future movement of cryptocurrencies and stocks**

It is known that social media platforms like twitter plays an imortant role in determining the price of financial asset platforms like stock market and crytpocurrencies. It is expected to increase the price of Doge after a tweet from Elon musk. Here the sentiment of public about a cryptocoin or stock (or any current issue going on!) could be calculated with the help of **twitter api** and **huggingface transformers** and a little mathematics

In [1]:
# modules for twitter api manipulation
!pip install --upgrade tweepy
import tweepy
import json
import pandas,numpy

In [18]:
# import necassery modules for mood analysis
!pip -q install transformers
!pip3 install emoji
import emoji
import transformers
from transformers import pipeline

sentimentmodel=pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")



Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

In [3]:
# authenticate the api key and token with twitter

from tweepy.auth import OAuthHandler

auth = OAuthHandler('t6d8uj4w9P6DjrNSmfSd42gEI', 'ZLcVXunCbN6NO4rUup6vTR33rO32epm0LFHLkzFLZnhjbQmQzZ')
auth.set_access_token('1405483945379074054-ktOMmQ6HUcZwzOxiHfjcFmP74hkTn5', 'hV792qfEAuJcvZmkzmoMf61qaaSnoV8D2YiIFYjzAgr06')


api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
# extract the tweets with the given keyword

def tweextractor(keyword):

  tweet=[]      #list to store tweets extracted
 
  # search_tweets method extract atmost 100 tweets with given keyword
  # use it with cursor to extract more than that in one go
  
  # take 500 popular tweets and 500 recent tweets to keep consistency in the result

  for i in tweepy.Cursor(api.search_tweets, keyword,tweet_mode="extended",lang='en',result_type='recent',count=100).items(500):
    i = json.dumps(i._json)
    i = json.loads(i)
    if i['full_text'] not in tweet:
      tweet.append(i['full_text'])
  for i in tweepy.Cursor(api.search_tweets, keyword,tweet_mode="extended",lang='en',result_type='popular',count=100).items(500):
    i = json.dumps(i._json)
    i = json.loads(i)
    if i['full_text'] not in tweet and len(i['full_text'])<500:
      tweet.append(i['full_text'])
  print(len(tweet),'tweets total tweets found')
  return tweet


In [5]:
#check tweextractor function

tweets=tweextractor('bitcoin')
tweets[:5]

441 tweets total tweets found


['RT @Rakontoken: Vitalik Buterin Isn’t Crazy About NFTs\n\nVitalik Buterin – known for his role in creating Ethereum, the largest altcoin on t…',
 'RT @BKEXGlobal: #BKEX New Listing | @bridge_oracle\n\n#BKEX will list #BRG/USDT🚀 🚀\n- Token type: BEP20\n- Deposit: 23:00 on Mar 13 (UTC+8)\n- T…',
 'Current Bitcoin (BTC-USD) price: 40110.668 USD                     (ID: 23593) #crypto #cryptocurrency #bitcoin #btc',
 'RT @YXorxx: @MemecoinMessiah @hpos10idotcom @dogecoin The ALPHA and the OMEGA.\nThe BEGINNING and the END\n\n#Bitcoin \n#cryptotrading https://…',
 'When I ask younger people what their long term plan is, they simply say they want to be rich... Yet they have no interest in economics, math, or #Bitcoin. \n\nThey also have no plan. \n\nStrange. \n\nWe are not the same.']

In [6]:
# method to take tweets and find overwhole sentiment to analyse where the
def sentimentanalyser(tweet):
  score=[]
  mood=sentimentmodel(tweet)
  for i in mood:
    if i['label']=='POS':
      score.append(i['score'])
    elif i['label']=='NEG':
      score.append(-i['score'])
  sentiment=sum(score)/len(score)
  if sentiment>0:
    print('Sentiment of public is positive with probability',sentiment)
  else:print('Sentiment of public is negative with probability',sentiment)
  

In [7]:
keyword='HDFC'    # check mood of public about the price of hdfc stock
tweets=tweextractor(keyword)
sentimentanalyser(tweets)

295 tweets total tweets found
Sentiment of public is negative with probability -0.5191410990624592


In [8]:
keyword='nifty50'    # check mood of public about the price of hdfc stock
tweets=tweextractor(keyword)
sentimentanalyser(tweets)

440 tweets total tweets found
Sentiment of public is positive with probability 0.27697749195560334
